In [1]:
# this part of the codes fetches requirements
import hashlib
from phe import paillier
import random as rd
import numpy as np

In [2]:
#  this part of the code creates unique BID for all participants
listOfIDS = []
numberOfBidders = 10
for i in range(0,numberOfBidders):
  i = str(i)
  listOfIDS.append(hashlib.sha256(i.encode('utf-8')).hexdigest())
i = 0

In [3]:
#this part creates the key pairs for all participants
dictionaryOfPublicKeys = {}
dictionaryOfPrivateKeys = {}
publicKeyAuc, privateKeyAuc = paillier.generate_paillier_keypair(n_length= 32)
for i in range(0, numberOfBidders):
    x,y = paillier.generate_paillier_keypair(n_length=64)
    dictionaryOfPublicKeys[listOfIDS[i]],dictionaryOfPrivateKeys[listOfIDS[i]]  = x ,y
i = 0

end of the Registration phase and start of Bid Submission Phase

In [17]:
# this part of the code generate random number and distinct bid values for all bidders and Auctioneer
dictionaryOfRandomNumbers = {}
dictionaryOfBids = {}
bids  = rd.sample(range(1, 100 ), numberOfBidders)
for i in range(0,numberOfBidders):
    r  = rd.randint(100,10000)
    dictionaryOfRandomNumbers[listOfIDS[i]] =  r
    dictionaryOfBids[listOfIDS[i]] = bids[i]
i = 0
print(dictionaryOfBids)
print(dictionaryOfRandomNumbers)


{'5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9': 28, '6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b': 7, 'd4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35': 67, '4e07408562bedb8b60ce05c1decfe3ad16b72230967de01f640b7e4729b49fce': 44, '4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328cb08b5531fcacdabf8a': 21, 'ef2d127de37b942baad06145e54b0c619a1f22327b2ebbcfbec78f5564afe39d': 69, 'e7f6c011776e8db7cd330b54174fd76f7d0216b612387a5ffcfb81e6f0919683': 85, '7902699be42c8a8e46fbbb4501726517e86b22c56a189f7625a6da49081b2451': 41, '2c624232cdd221771294dfbb310aca000a0df6ac8b66b696d90ef06fdefb64a3': 20, '19581e27de7ced00ff1ce50b2047e7a567c76b1cbaebabe5ef03f7c3017bb5b7': 6}
{'5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9': 5710, '6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b': 5761, 'd4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35': 5105, '4e07408562bedb8b60ce05c1decfe3ad16b72230967de01f640b7e472

In [5]:
# this PART OF THE CODE SIMULATES BIDDER GENERATING PRODUCT AND THAN SENDING TO NEXT BIDDER
finalSubmittedDataFirstPhase = [] #will be sused to stored the final resuklt of the two bidder for bid submission
for i in range(0,numberOfBidders):
    id = listOfIDS[i]
    dataPerBidder= []
    # print(dictionaryOfBids[id] * dictionaryOfRandomNumbers[id])
    encrytionOfBidAndRandomNumber = publicKeyAuc.raw_encrypt(dictionaryOfBids[id] * dictionaryOfRandomNumbers[id])
    # this part of code is forwarding the encrytionOfBidAndRandomNumber to all other bidders and after multiplication of the random number of the next
    # bidder it is sent to the Auctioneer immediately
    for j in range(0,numberOfBidders):
        if j != i:
            idOfNextBidder = listOfIDS[j]
            pubKeyNextBidder, priKeyNextBidder= dictionaryOfPublicKeys[idOfNextBidder],dictionaryOfPrivateKeys[idOfNextBidder]
            e = pubKeyNextBidder.raw_encrypt(encrytionOfBidAndRandomNumber)
            e = priKeyNextBidder.raw_decrypt(e)
            encrytionOfBidAndBothRandomNumber = e ** dictionaryOfRandomNumbers[idOfNextBidder]
            # this part of the code simulates the storing of the data of the bid summision to auctioneer data
            dataPerBidder.append(privateKeyAuc.raw_decrypt(encrytionOfBidAndBothRandomNumber))
        else:
            dataPerBidder.append(0)
    finalSubmittedDataFirstPhase.append(dataPerBidder)
i,j = 0,0

In [6]:
for i in range(len(finalSubmittedDataFirstPhase)):
    print(finalSubmittedDataFirstPhase[i])
i = 0

[0, 4105920, 3756480, 4298112, 6744192, 5101824, 1738464, 1843296, 3765216, 3214848]
[366600, 0, 1010500, 1156200, 1814200, 1372400, 467650, 495850, 1012850, 864800]
[2012400, 6063000, 0, 6346800, 9958800, 7533600, 2567100, 2721900, 5559900, 4747200]
[6523920, 19655400, 17982600, 0, 32285040, 24422880, 8322180, 8824020, 18024420, 15389760]
[2649504, 7982480, 7303120, 8356128, 0, 9918656, 3379816, 3583624, 7320104, 6250112]
[1822080, 5489600, 5022400, 5746560, 9016960, 0, 2324320, 2464480, 5034080, 4298240]
[1769508, 5331210, 4877490, 5580756, 8756796, 6624312, 0, 2393373, 4888833, 4174224]
[3127020, 9421150, 8619350, 9862140, 15474740, 11706280, 3988955, 0, 8639395, 7376560]
[1008540, 3038550, 2779950, 3180780, 4990980, 3775560, 1286535, 1364115, 0, 2379120]
[4018560, 12107200, 11076800, 12673920, 19886720, 15043840, 5126240, 5435360, 11102560, 0]


implementation of the second phase here auctioneer reforward each packet to every bidder to multiply their random number.

In [7]:
def power(val, power):
    result = pow(val, power//2)
    result = result * result
 
    if power % 2 != 0:
        result = result * val
    return result

In [12]:
#this function works as the intermediate steps where we are requesting each bidder to multiplyt his random number to the given ciphertext
productOfRandomNumbers = 0
for k in range(numberOfBidders):
    productOfRandomNumbers = productOfRandomNumbers + dictionaryOfRandomNumbers[listOfIDS[k]]
productOfRandomNumbers

4113

In [15]:
print(pow(publicKeyAuc.g,156))

3432463116501949859470350356630567201616430776202747530266213102264495834374557499543104314091801777767520862027305090220697469391469563139799673829353764254364474843089701232029216376273941093955532673906580272139817543517294668259852279777809718169344576887437404700258264835123694538848342581795952369210135596842378844232997411137311210792744348697172715279500469783460316855431316939801689289447443008076591164073474880889489810686658203226709903727679951140308100868042850882435454246069622236108760480525573345747201545446613439553016782858408670804481156191858306639982356957543693745568134828856854027596125826218127140510827017960677781181152200307455204203146990407602033959737191322179794293382860243005249520151795870661469986225125186511323716472913786604608068932273599100903155689338062595800664249909667662683138049007873351484243065115914791440699539698492181382570486711879065457858400139644764243895922843719467093841032102626003814851632592841645722651175710883050793738554990475

In [10]:
def everyBiddersRandomNumberMultiplication(x):
    X = np.power(x,productOfRandomNumbers)
    return x

In [11]:
finalDataOfSecondPhase = []
auctioneerRandomNumber = rd.randint(2,2)
print(auctioneerRandomNumber)
for i in range(len(finalSubmittedDataFirstPhase)):
    dataPerBidder = []
    for j in range(numberOfBidders):
        if finalSubmittedDataFirstPhase[i][j] == 0 :
            dataPerBidder.append(0)
        else:
            encryptedValueByAuctioneer = publicKeyAuc.raw_encrypt(finalSubmittedDataFirstPhase[i][j] * auctioneerRandomNumber)
            returnValue = everyBiddersRandomNumberMultiplication(encryptedValueByAuctioneer)
            dataPerBidder.append((privateKeyAuc.raw_decrypt(returnValue)//auctioneerRandomNumber))
    finalDataOfSecondPhase.append(dataPerBidder)
i,j = 0,0


2


KeyboardInterrupt: 

In [ ]:

for i in range(len(finalDataOfSecondPhase)):
    print(finalDataOfSecondPhase[i])
i = 0

[0, 269095500, 203847075, 124709850, 212672250, 143951625, 270108225, 77979825, 68431275, 45427950]
[78934680, 0, 57656280, 35273040, 60152400, 40715400, 76397640, 22055880, 19355160, 12848880]
[62513103, 60277020, 0, 27934834, 47638290, 32244965, 60503869, 17467373, 15328511, 10175798]
[156303012, 150712080, 114168452, 0, 119111160, 80622860, 151279276, 43674092, 38326244, 25442792]
[82233270, 79291800, 60065670, 36747060, 0, 42416850, 79590210, 22977570, 20163990, 13385820]
[136274205, 131399700, 99538805, 60895990, 103848150, 0, 131894215, 38077655, 33415085, 22182530]
[183673593, 177103620, 134160753, 82077054, 139968990, 94740915, 0, 51321963, 45037641, 29898138]
[16635696, 16040640, 12151216, 7433888, 12677280, 8580880, 16101008, 0, 4079152, 2707936]
[86679615, 83579100, 63313415, 38733970, 66054450, 44710325, 83893645, 24219965, 0, 14109590]
[44216538, 42634920, 32297098, 19758764, 33695340, 22807390, 42795374, 12354958, 10842106, 0]


In [19]:
d = publicKeyAuc.raw_encrypt(10)
d = privateKeyAuc.raw_decrypt(d*pow(publicKeyAuc.g,5105))
print(d)

5115
